In [1]:
import pandas as pd

In [20]:
outages2022 = pd.read_csv("/Users/daniel_personal/Downloads/eaglei_outages/eaglei_outages_2022.csv")

### Exploring whether I could use less-than-yearly granularity data from wildfires – no

In [21]:
outages2022[outages2022["county"] == "Pueblo"]

,fips_code,county,state,sum,run_start_time
621,8101,Pueblo,Colorado,39.0,2022-01-01 00:15:00
1153,8101,Pueblo,Colorado,189.0,2022-01-01 00:30:00
1674,8101,Pueblo,Colorado,189.0,2022-01-01 00:45:00
2201,8101,Pueblo,Colorado,189.0,2022-01-01 01:00:00
2724,8101,Pueblo,Colorado,189.0,2022-01-01 01:15:00
...,...,...,...,...,...
22240423,8101,Pueblo,Colorado,132.0,2022-11-11 18:00:00
22243588,8101,Pueblo,Colorado,12.0,2022-11-11 18:45:00
22244692,8101,Pueblo,Colorado,12.0,2022-11-11 19:00:00
22245725,8101,Pueblo,Colorado,12.0,2022-11-11 19:15:00


In [4]:
import json
import geopandas as gpd
from tqdm import tqdm
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
# pre-processed data containing distances
with open("../../intermediate_data/fire_features_with_distances.json", "r") as file:
    feature_list = json.load(file)
# shortest distances got saved as a list instead of a float
# transform them to floats
for feature in feature_list:
    feature["attributes"]["shortest_distance_to_pueblo"] = feature["attributes"]["shortest_distance_to_pueblo"][0]
    
def convert_to_geodataframe(feature_list):
    
    # store processed features
    processed_features = []

    for i, feature in tqdm(enumerate(feature_list)):
        attributes = feature['attributes']
        geometry = feature.get('geometry')

        # some geometries are not rings; this can be problematic, so
        # I add a check and log exceptions
        if geometry and 'rings' in geometry:
            try:
                polygon = Polygon(geometry['rings'][0])
                processed_features.append({**attributes, 'geometry': polygon})

            except Exception as e:
                print(f"Problem with geometry for OBJECTID {
                      attributes.get('OBJECTID')}: {e}")

        elif geometry and 'curveRings' in geometry:
            try:
                polygon = Polygon(geometry['curveRings'][0])
                processed_features.append({**attributes, 'geometry': polygon})

            except Exception as e:
                print(f"Problem with geometry for OBJECTID {
                      attributes.get('OBJECTID')}: {e}")
        else:
            print(f"Skipping feature with OBJECTID {attributes.get(
                'OBJECTID')}: No 'rings' found. Instead, got: {geometry}\n")

    gdf = gpd.GeoDataFrame(processed_features, geometry='geometry')
    return gdf
gdf = convert_to_geodataframe(feature_list)

111987it [00:12, 13369.12it/s]

Problem with geometry for OBJECTID 109605: could not convert string to float: 'b'
Problem with geometry for OBJECTID 110224: could not convert string to float: 'a'
Problem with geometry for OBJECTID 110639: could not convert string to float: 'b'
Problem with geometry for OBJECTID 111431: could not convert string to float: 'a'
Problem with geometry for OBJECTID 111897: could not convert string to float: 'b'
Problem with geometry for OBJECTID 112410: could not convert string to float: 'a'
Problem with geometry for OBJECTID 112415: could not convert string to float: 'a'


115418it [00:12, 15181.65it/s]

Problem with geometry for OBJECTID 113411: could not convert string to float: 'a'
Problem with geometry for OBJECTID 113665: could not convert string to float: 'a'
Problem with geometry for OBJECTID 113738: could not convert string to float: 'a'
Problem with geometry for OBJECTID 113766: could not convert string to float: 'a'
Problem with geometry for OBJECTID 113805: could not convert string to float: 'a'
Problem with geometry for OBJECTID 114309: could not convert string to float: 'a'
Problem with geometry for OBJECTID 114322: could not convert string to float: 'a'
Problem with geometry for OBJECTID 115629: could not convert string to float: 'a'
Problem with geometry for OBJECTID 115974: could not convert string to float: 'a'
Problem with geometry for OBJECTID 116235: could not convert string to float: 'c'


120731it [00:13, 17235.43it/s]

...

124309it [00:13, 16702.63it/s]

Problem with geometry for OBJECTID 121010: could not convert string to float: 'a'
Problem with geometry for OBJECTID 122264: could not convert string to float: 'a'


127644it [00:13, 16130.64it/s]

Problem with geometry for OBJECTID 125745: could not convert string to float: 'a'
Problem with geometry for OBJECTID 127492: could not convert string to float: 'a'


135061it [00:13, 9721.52it/s] 


In [10]:
gdf.columns

Index(['OBJECTID', 'USGS_Assigned_ID', 'Assigned_Fire_Type', 'Fire_Year',
       'Fire_Polygon_Tier', 'Fire_Attribute_Tiers', 'GIS_Acres',
       'GIS_Hectares', 'Source_Datasets', 'Listed_Fire_Types',
       'Listed_Fire_Names', 'Listed_Fire_Codes', 'Listed_Fire_IDs',
       'Listed_Fire_IRWIN_IDs', 'Listed_Fire_Dates', 'Listed_Fire_Causes',
       'Listed_Fire_Cause_Class', 'Listed_Rx_Reported_Acres',
       'Listed_Map_Digitize_Methods', 'Listed_Notes', 'Processing_Notes',
       'Wildfire_Notice', 'Prescribed_Burn_Notice', 'Wildfire_and_Rx_Flag',
       'Overlap_Within_1_or_2_Flag', 'Circleness_Scale', 'Circle_Flag',
       'Exclude_From_Summary_Rasters', 'Shape_Length', 'Shape_Area',
       'shortest_distance_to_pueblo', 'centroid_distance_to_pueblo',
       'geometry'],
      dtype='object')

In [13]:
[x for x in (
    gdf
    .loc[(gdf["centroid_distance_to_pueblo"] < 100) & (gdf["Assigned_Fire_Type"] == "Wildfire") & (gdf["Fire_Year"] > 2014)]["Listed_Fire_Dates"]
)]

['Listed Wildfire Discovery Date(s): 2015-12-31 (1) | Listed Other Fire Date(s): 2016-11-21 - NIFC DATE_CUR field (1) | Listed Upload Date(s): 2016-11-21 (1)',
 'Listed Wildfire Discovery Date(s): 2015-04-12 (1) | Listed Other Fire Date(s): 2018-06-20 - NIFC DATE_CUR field (1), 2015-04-13 - DATE_CUR field (1), 2018-06-20 - REVDATE field (1)',
 'Listed Wildfire Discovery Date(s): 2015-09-05 (1) | Listed Other Fire Date(s): 2018-06-20 - NIFC DATE_CUR field (1), 2018-06-20 - REVDATE field (1)',
 'Listed Wildfire Discovery Date(s): 2015-12-31 (1) | Listed Other Fire Date(s): 2016-11-21 - NIFC DATE_CUR field (1) | Listed Upload Date(s): 2016-11-21 (1)',
 'Listed Ignition Date(s): 2016-10-17 (1) | Listed Wildfire Discovery Date(s): 2016-10-17 (2) | Listed Other Fire Date(s): 2017-01-13 - NIFC DATE_CUR field (1), 2018-06-20 - NIFC DATE_CUR field (1), 2016-12-15 - DATE_CUR field (1), 2018-06-20 - REVDATE field (1) | Listed Upload Date(s): 2017-01-13 (1)',
 'Listed Ignition Date(s): 2016-07-08 

### Exploring Colorado Asthma Dataset

In [23]:
asthma_df = pd.read_excel("/Users/daniel_personal/Downloads/TEEO_REF_CDPHE COEPHT Asthma Data 2011-2022_2023_EN.xlsx", 3)

In [25]:
pueblo_asthma_cases = (
    asthma_df
    .loc[(asthma_df["COUNTY"] == "Pueblo") & (asthma_df["GENDER"] == "Both genders")]
)

In [26]:
pueblo_asthma_cases

,COUNTY,RATE,L95CL,U95CL,HOSP,FLAG,YEAR,GENDER,MEASURE,HEALTHOUTCOMEID,cofips
2966,Pueblo,12.767398,10.955735,14.579060,193.0,3,2004,Both genders,Age adjusted rate,Asthma,8101
2969,Pueblo,15.275842,13.313133,17.238551,237.0,3,2005,Both genders,Age adjusted rate,Asthma,8101
2972,Pueblo,14.870266,12.943126,16.797405,233.0,3,2006,Both genders,Age adjusted rate,Asthma,8101
2975,Pueblo,11.532079,9.843563,13.220595,183.0,3,2007,Both genders,Age adjusted rate,Asthma,8101
2978,Pueblo,13.711419,11.893815,15.529024,224.0,3,2008,Both genders,Age adjusted rate,Asthma,8101
2981,Pueblo,14.447722,12.549797,16.345647,231.0,3,2009,Both genders,Age adjusted rate,Asthma,8101
2984,Pueblo,12.375306,10.645314,14.105299,204.0,3,2010,Both genders,Age adjusted rate,Asthma,8101
2987,Pueblo,10.687871,9.064611,12.311130,174.0,3,2011,Both genders,Age adjusted rate,Asthma,8101
2990,Pueblo,8.107688,6.682950,9.532425,131.0,2,2012,Both genders,Age adjusted rate,Asthma,8101
2993,Pueblo,6.834254,5.543627,8.124881,114.0,2,2013,Both genders,Age adjusted rate,Asthma,8101


In [27]:
(
    gdf
    .loc[(gdf["centroid_distance_to_pueblo"] < 100) & (gdf["Assigned_Fire_Type"] == "Wildfire") & (gdf["Fire_Year"] >= 2004)]
)

,OBJECTID,USGS_Assigned_ID,Assigned_Fire_Type,Fire_Year,Fire_Polygon_Tier,Fire_Attribute_Tiers,GIS_Acres,GIS_Hectares,Source_Datasets,Listed_Fire_Types,...,Wildfire_and_Rx_Flag,Overlap_Within_1_or_2_Flag,Circleness_Scale,Circle_Flag,Exclude_From_Summary_Rasters,Shape_Length,Shape_Area,shortest_distance_to_pueblo,centroid_distance_to_pueblo,geometry
47699,47700,47700,Wildfire,2004,1,"1 (2), 3 (2)",1173.671717,474.968093,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (4),...,None,None,0.556983,NaN,No,10351.808951,4.749681e+06,39.242773,40.388470,"POLYGON ((-762461.683 -205816.795, -762540.847..."
47755,47756,47756,Wildfire,2004,1,"1 (2), 3 (2)",500.224308,202.433596,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (3), Likely Wildfire (1)",...,None,"Caution, this Wildfire in 2004 overlaps with a...",0.730242,NaN,No,5902.185072,2.024336e+06,55.564276,56.059262,"POLYGON ((-765078.557 -99783.946, -764985.465 ..."
48706,48707,48707,Wildfire,2005,1,"1 (2), 2 (1), 3 (5), 4 (3)",11368.900073,4600.830628,Comb_National_NIFC_Interagency_Fire_Perimeter_...,"Wildfire (4), Likely Wildfire (7)",...,None,None,0.488010,NaN,No,34419.828442,4.600831e+07,22.155022,24.911368,"POLYGON ((-749287.077 -176991.734, -749269.472..."
49042,49043,49043,Wildfire,2005,1,"1 (5), 3 (5)",1285.438753,520.198607,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (10),...,None,None,0.246554,NaN,No,16282.950645,5.201986e+06,40.481585,40.481636,"POLYGON ((-762375.406 -208309.786, -762375.38 ..."
49054,49055,49055,Wildfire,2005,1,"1 (1), 3 (1)",390.636100,158.084821,Comb_National_NIFC_Interagency_Fire_Perimeter_...,Wildfire (2),...,None,None,0.155668,NaN,No,11296.680235,1.580848e+06,49.838183,50.416842,"POLYGON ((-669561.591 -242924.946, -669552.897..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97048,97049,97049,Wildfire,2020,3,3 (1),0.066696,0.026991,Comb_National_NIFC_Wildland_Fire_Perimeters_FH...,Wildfire (1),...,None,None,0.756955,NaN,No,66.938730,2.699075e+02,56.630367,56.637748,"POLYGON ((-731504.925 -78666.297, -731504.921 ..."
97086,97087,97087,Wildfire,2020,3,3 (2),0.058978,0.023868,Comb_National_NIFC_Wildland_Fire_Perimeters_FH...,Wildfire (2),...,None,None,0.500399,NaN,No,77.419550,2.386753e+02,66.417055,66.422055,"POLYGON ((-742548.691 -65299.059, -742552 -653..."
97096,97097,97097,Wildfire,2020,3,3 (1),0.057385,0.023223,Comb_National_NIFC_Wildland_Fire_Perimeters_FH...,Wildfire (1),...,None,None,0.474428,NaN,No,78.428990,2.322274e+02,51.750467,51.754531,"POLYGON ((-787254.524 -159502.008, -787257.423..."
97098,97099,97099,Wildfire,2020,3,3 (1),0.056553,0.022886,Comb_National_NIFC_Wildland_Fire_Perimeters_FH...,Wildfire (1),...,None,None,0.761511,NaN,No,61.454226,2.288601e+02,59.902092,59.909344,"POLYGON ((-729728.186 -72385.888, -729725.452 ..."
